In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep
import time
from multiprocessing import Pool

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-01-25 11:23:30.409048


In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
albs = albums(disc)
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


# Download Artist Data

In [ ]:
artistIDToRef = disc.getArtistIDToRefData()

In [ ]:
myMusicAlbumIDMap = getFile("/Users/tgadfort/Documents/code/music/musicDiscogsMap.p")

In [ ]:
for artistName,artistVals in myMusicAlbumIDMap.items():
    artistID,discogsName = artistVals
    #arts.rmIDFiles(artistID)
    #artistID = '12596' # James Brown
    #ref      = artistIDToRef['12596']
    #arts.downloadArtistFromID(artistID, ref)

In [ ]:
artistIDs = [v[0] for k,v in myMusicAlbumIDMap.items()]
arts.rmIDsFromDBs(artistIDs)

In [ ]:
arts.parseArtistModValFiles(96, force=False)
arts.parseArtistModValExtraFiles(96)

In [ ]:
files = [savename]
from glob import glob
from os.path import join
from fileUtils import getDirname
files += glob(join(getDirname(savename), "extra", "{0}-*.p".format(artistID)))
files

# Download Missing Artists

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
from discogsBase import discogs

disc  = discogs()
albs  = albums(disc)
toget = albs.findMissingArtistsFromAlbums()

from ioUtils import saveFile
saveFile(idata=toget, ifile="missingArtistsFromAlbums.p")

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)

from ioUtils import getFile

toget = getFile(ifile="missingArtistsFromAlbums.p")

from time import sleep
togetmap  = toget['togetmap']
refstoget = toget['toget']
for ia,item in enumerate(refstoget.most_common()):
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            print('\n{0}/{1}'.format(ia,len(refstoget)))
            retval = arts.downloadArtistURL(url, savename, debug=True)
            sleep(3)
        except:
            continue
    else:
        print("{0} known".format(savename))
            
    #print(item)
#len(togetmap)

In [ ]:
arts.rmIDFromDB(artistID)

In [ ]:
del db[artistID]

In [ ]:
disc.saveArtistsDBModValData(96, db)

In [ ]:
arts.parseArtistModValFiles(96, force=False)

In [ ]:
arts.assertDBModValExtraData(96)

In [ ]:
#arts.parseArtistModValFiles(96, force=False)

# Missing Artists From Albums

In [24]:
%load_ext autoreload
%autoreload
from albums import albums
from discogsBase import discogs

disc  = discogs()
albs  = albums(disc)
toget = albs.findMissingArtistsFromAlbums()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists
Finding Missing Artists From Downloaded Albums
Loading data from /Volumes/Music/Discog/db/ArtistIDToName.p
  --> This file is 13.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToName.p
	 /Volumes/Music/Discog/albums-db/metadata/0-ArtistMetadata.p 	 458
	 /Volumes/Music/Discog/albums-db/metadata/1-ArtistMetadata.p 	 924
	 /Volumes/Music/Discog/albums-db/metadata/10-ArtistMetadata.p 

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)

from time import sleep
togetmap  = toget['togetmap']
refstoget = toget['toget']
for ia,item in enumerate(refstoget.most_common()):
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            print('\n{0}/{1}'.format(ia,len(refstoget)))
            retval = arts.downloadArtistURL(url, savename, debug=True)
            sleep(2)
        except:
            continue
    else:
        print("{0} known".format(savename))
            
    #print(item)
#len(togetmap)

# Artists From Search

In [26]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
arts.searchDiscogForArtist("Meathook Seed")
#arts.searchDiscogForArtist("Continental Divide")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


===================== Searching For Meathook Seed =====================
Downloading: https://www.discogs.com/search/?q=Meathook%20Seed&limit=250&type=artist
Found 2 artists
2 / 2 	: 6 	 https://www.discogs.com/artist/300089-Meathook-Seed?sort=year%2Casc&limit=500
  Downloaded 0 Additional URLs
Saving /Volumes/Music/Discog/artists/89/300089.p (force=False)
  --> This file is 18.3

In [ ]:
from ioUtils import getFile

In [ ]:
toGet = getFile(ifile="/Users/tgadfort/Documents/code/charts/chartCounter.p", debug=True)

In [ ]:
toGet = getFile("~/code/charts/toget.p")

In [ ]:
for i,val in enumerate(toGet):
    if i <= 206:
        continue
    print('='*30,val,'(',i,'/',len(toGet),')','='*30)
    arts.searchDiscogForArtist(val)
    print("\n\n\n")
    sleep(2)

In [ ]:
from time import sleep
vals  = ["The Souvenirs", "The Stand", f"The Suicide Kings", "The Supremes", "The Temptations", "The Tomcats", "The Tone", "The Tourists", "The Unseen", "The Ventures", "The Vikings", "The Violents", "The Wailers", "The Want", "The Washingtonians", "The Witches", "UB40", "Underdog", "Unearth", "Unk", "Unscarred", "Up Front", "Up In Arms", "Urban Legends"]
vals += ["The Pits", "The Pressure", "The Pretenders", "The Prodigy", "The Product", "The Receptionists", "The Retardos", "The Revelers", "The Romancers", "The Roots", "The Rosehips", "The Ruiners", "The Runaways", "The Scholars", "The Seeds", "The Shams", "The Sights", "The Sillies", "The Skunks", "The Snowmen"]
vals += ["The Invisible Men", "The Israelites", "The Ivory Coast", "The Killers", "The Manatees", "The Marquees", "The Masters", "The Matrix", "The Mixtures", "The Moves", "The Noisettes", "The Nomads", "The Orange Peels", "The Outcasts"]
vals += ["The Game", "The Generics", "The Gits", "The Go-Betweens", "The Goblins", "The Godz", "The Great Society", "The Haters", "The Hilltops", "The Hookers", "The Immortals", "The Impacts", "The Inspector", "The Invaders"]
vals += ["The Diamonds", "The Diplomats", "The Dramatics", "The Drifters", "The Dust Brothers", "The Eagles", "The Exceptions", "The Fiends", "The Fixtures", "The Frantics", "The Freeze", "The Frogs", "The Fumes"]
vals += ["The Cardigans", "The Casualties", "The Challengers", "The Chameleons", "The Cheaters", "The Chicken Hawks", "The Crash", "The Crusaders", "The Cure", "The Darts", "The Deuce Coupes"]
vals += ["The Beat", "The Beatles", "The Blood Brothers", "The Blow Up", "The Bollweevils", "The Brentwoods", "The Broadways", "The Brood", "The Business", "The Busy Signals"]
vals += ["You", "Young Money", "Young Pioneers", "Zero Zero"]
for val in vals:
    print('='*30,val,'='*30)
    try:
        arts.searchDiscogForArtist(val)
    except:
        continue
    print("\n\n\n")
    sleep(10)

# New Artists From Albums

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums

disc = discogs()
albs = albums(disc)
toget = albs.findMissingArtistsFromAlbums()

In [ ]:
from time import sleep
togetmap  = toget['togetmap']
refstoget = toget['toget']
for ia,item in enumerate(refstoget.most_common()):
    print('\n{0}/{1}'.format(ia,len(refstoget)))
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            arts.downloadArtistURL(url, savename, debug=True)
        except:
            continue
        sleep(2)
    #print(item)
#len(togetmap)

# Albums From Known People

In [ ]:
data = getFile("/Volumes/Music/MusicDB/toget.p")

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
discdf = disc.getMasterDiscogsDB()
print(discdf.shape)
discdf = discdf[~discdf['Name'].isna()]
print(discdf.shape)

artists = list(discdf["Artist"])

_ = clock("Last Run")

In [ ]:
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None
    
toget={}
known={}

for ia,artistName in enumerate(data):
    #artistID = artistName
    artistID = None
    if artistID is None:
        retval = getMusicData('Name', artistName)
        if retval is not None:
            artistID = retval.index.values[0]
    modValue  = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
    if toget.get(modValue) is None:
        toget[modValue] = {}
    toget[modValue][artistID] = True
        
    if ia % 100 == 0:
        print(ia,'/',len(data))

In [ ]:
for modVal in toget.keys():
    artistIDs = toget[modVal]
    print(modVal,'\t',len(artistIDs))
    dbdata = disc.getArtistsDBModValData(modVal)
    for artistID in artistIDs:
        print(' ',artistID)
        artistData = dbdata[artistID]
        albs.downloadAlbumFromArtistData(artistID=artistID, artistData=artistData, mediaTypes=None)

# Albums From All Artists

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=1, mediaTypes=None, rand=True)

# Artists From Collections

In [ ]:
%load_ext autoreload
%autoreload

from collection import Counter
refCounts = Counter(disc.getCollectionRefCountsData())

In [ ]:
artistIDDF = disc.getMasterArtistDiscogsDB()
print(artistIDDF["Ref"].shape)
artistRefs = {v: k for k, v in artistIDDF["Ref"].to_dict().items()}
artistIDs  = {k: v for k, v in artistIDDF["Ref"].to_dict().items()}
print(len(artistRefs))

In [ ]:
try:
    errFile = setFile(disc.getDiagnosticDir(), "CollectionErrors.json")
    errs = getFile(errFile)
except:
    raise ValueError("Problem loading collections file")
    
print("Found {0} errors".format(len(errs)))
nNext = len(errs)

n = len(refCounts)
nFalse = 0
nNone  = 0
nTrue  = 0

for i,(ref,cnts) in enumerate(refCounts.most_common()):
    if i < 286093:
        continue
    if i % 2500 == 0:
        print("Processed {0}/{1}  (Errs={2})".format(i,n, len(errs)))
        
    if errs.get(ref) is not None:
        continue
        
    artistID = dutils.getArtistID(ref)
    if artistID is None:
        continue
    if artistIDs.get(artistID) is None:
        url      = arts.getArtistURL(ref)
        savename = arts.getArtistSavename(artistID)
        if isFile(savename):
            continue
            
        try:
            retval = arts.downloadArtistURL(url=url, savename=savename, parse=True, debug=True)
            sleep(1)
        except:
            retval = None
            
        if retval is None:
            nNone += 1
            errs[ref] = "None"
            print("\t--->Adding None for {0}".format(artistID))
        if retval is False:
            nFalse += 1
            errs[ref] = "False"
            print("\t--->Adding False for {0}".format(artistID))

            
        sleep(3)

    if i % 1000 == 0:
        print("There have been {0} errors so far...".format(nNone))
        print("There have been {0} no saves so far...".format(nFalse))
        print("There have been {0} full saves so far...".format(nTrue))
        
        errFile = setFile(disc.getDiagnosticDir(), "CollectionErrors.json")
        saveFile(ifile=errFile, idata=errs, debug=True)


            
    if len(errs) == nNext + 100:
        errFile = setFile(disc.getDiagnosticDir(), "CollectionErrors.json")
        saveFile(ifile=errFile, idata=errs, debug=True)
        print("Breaking after 250 errors")
        print("Last I = {0}".format(i))
        break

In [ ]:
len(errs)

In [ ]:
disc.getDiagnosticDir()

In [ ]:
# All Music

In [ ]:
#https://www.allmusic.com/search/all/Bliss
#

In [ ]:
savename="bliss.p"

In [ ]:
url = "https://www.allmusic.com/search/artists/Bliss"

In [ ]:
def getAllMusic(url, savename, parse=False, force=False, debug=False, sleeptime=2):
    if isFile(savename):
        if debug:
            print("{0} exists.".format(savename))
        if force is False:
            return False
        else:
            print("Downloading again.")

    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    if debug:
        print("Now Downloading in Artists(): {0}".format(url))
        print("                   Saving as: {0}".format(savename))

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need

    print("Saving {0} (force={1})".format(savename, force))
    saveJoblib(data=data, filename=savename, compress=True)
    print("Done. Sleeping for {0} seconds".format(sleeptime))
    sleep(sleeptime)

    if isFile(savename):
        return True
    else:
        return False


In [ ]:
getAllMusic(url, savename)

In [ ]:
from webUtils import getHTML

In [ ]:
data = getHTML('bliss.p')

In [ ]:
data

In [ ]:
uls = data.findAll("ul", {"class": "search-results"})

In [ ]:
for ul in uls:
    for li in ul.findAll("li", {"class": "artist"}):
        for div in li.findAll("div", {"class": "name"}):
            link     = div.find("a")
            href     = link.attrs['href']
            tooltip  = link.attrs['data-tooltip']
            artistID = None
            if isinstance(tooltip, dict):
                artistID = tooltip['id']
            
#            print(div,'\n\n')

In [ ]:
%load_ext autoreload
%autoreload
discAM = discogs(base='allmusic')

In [ ]:
%load_ext autoreload
%autoreload
artsAM = artistsAM(discAM)

In [ ]:
artsAM.searchAllMusicForArtist('Creed')

In [ ]:
artistRef = "https://www.allmusic.com/artist/creed-warner-mn0003293932"
artistRef = "{0}/discography".format(artistRef)
artistRef
#urllib.parse.urljoin(artistRef) #, "/discography")

In [ ]:
from fsUtils import setFile, isFile, setDir, isDir, mkDir, mkSubDir, setSubFile
from fsUtils import removeFile
from fileUtils import getBasename, getBaseFilename
from ioUtils import getFile, saveFile, saveJoblib
from webUtils import getWebData, getHTML, getURL
from searchUtils import findExt, findPattern
from timeUtils import clock, elapsed, update
from collections import Counter
from math import ceil
from time import sleep
from artist import artist
from discogsUtils import allmusicUtils
import urllib
from urllib.parse import quote

class artistsAM():
    def __init__(self, discog, basedir=None):
        self.disc = discog
        self.name = "artists-allmusic"
        
        self.artist = artist()
        
        ## General Imports
        self.getCodeDir          = self.disc.getCodeDir
        self.getArtistsDir       = self.disc.getArtistsDir
        self.getArtistsDBDir     = self.disc.getArtistsDBDir
        self.getDiscogDBDir      = self.disc.getDiscogDBDir
        self.discogsUtils        = allmusicUtils()
        
        self.prevSearches        = {}
        
        self.modVal = self.disc.getMaxModVal
        
        self.artistIDtoRefData = None
        
        self.starterDir = setDir(self.getCodeDir(), self.name)
        if not isDir(self.starterDir):
            print("Creating {0}".format(self.starterDir))
            mkDir(self.starterDir, debug=True)
        
    
    ###############################################################################
    # Artist Data
    ###############################################################################
    def getData(self, ifile):
        info = self.artist.getData(ifile)
        return info
    
    def getFileData(self, artistID):
        ifile = self.getArtistSavename(artistID, 1)
        info  = self.getData(ifile)
        return info
        
    
    ###############################################################################
    # Artist Info
    ###############################################################################
    def getArtistURL(self, artistRef, page=1):
        
        url = "{0}/discography".format(artistRef)
        return url
        
        if artistRef.startswith("http"):
            return artistRef
        else:
            baseURL = self.disc.discogURL
            url     = urllib.parse.urljoin(baseURL, quote(artistRef))
            return url

            
        if artistRef.startswith("/artist/") is False:
            print("Artist Ref needs to start with /artist/")
            return None
        
        baseURL = self.disc.discogURL
        url     = urllib.parse.urljoin(baseURL, quote(artistRef))
        url     = urllib.parse.urljoin(url, "?sort=year%2Casc&limit=500") ## Make sure we get 500 entries)
        if isinstance(page, int) and page > 1:
            pageURL = "&page={0}".format(page)
            url = "{0}{1}".format(url, pageURL)
        return url
    
    
    def getArtistSavename(self, discID, page=1):
        artistDir = self.disc.getArtistsDir()
        modValue  = self.discogsUtils.getDiscIDHashMod(discID=discID, modval=self.disc.getMaxModVal())
        if modValue is not None:
            outdir    = mkSubDir(artistDir, str(modValue))
            if isinstance(page, int) and page > 1:
                outdir = mkSubDir(outdir, "extra")
                savename  = setFile(outdir, discID+"-{0}.p".format(page))
            else:
                savename  = setFile(outdir, discID+".p")
                
            return savename
        return None
        
    
    ###############################################################################
    # Artist Downloads
    ###############################################################################
    def downloadArtistURL(self, url, savename, parse=False, force=False, debug=False, sleeptime=2):
        if isFile(savename):
            if debug:
                print("{0} exists.".format(savename))
            if force is False:
                return False
            else:
                print("Downloading again.")
                        
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,} 

        if debug:
            print("Now Downloading in Artists(): {0}".format(url))
            print("                   Saving as: {0}".format(savename))

        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need

        if parse is True:
            info = self.artist.getData(str(data))
            ID = info.ID.ID
            
            if savename.find("/extra/") == -1:
                artistID = getBaseFilename(savename)
            else:
                artistID = getBaseFilename(savename).split("-")[0]
                if ID != artistID:
                    raise ValueError("Problem saving {0} with artistID {1}".format(savename, ID))
                
            if ID != artistID:
                print("  File ID != Artist ID. Renaming to {0}".format(savename))

                savename_test = self.getArtistSavename(ID)
                if force is False and isFile(savename_test):
                    print("{0} exists.".format(savename_test))
                    return False

            
        if force is False and isFile(savename):
            print("{0} exists.".format(savename))
            return True
            
            
        print("Saving {0} (force={1})".format(savename, force))
        saveJoblib(data=data, filename=savename, compress=True)
        print("Done. Sleeping for {0} seconds".format(sleeptime))
        sleep(sleeptime)
        
        if isFile(savename):
            return True
        else:
            return False
        
        
            
    ################################################################################
    # Download Search Artist (2a)
    ################################################################################
    def searchAllMusicForArtist(self, artist, debug=True, sleeptime=2):
        if self.prevSearches.get(artist) is not None:
            return
        if self.prevSearches.get(artist.upper()) is not None:
            return
        self.prevSearches[artist] = True
        
        
        print("\n\n===================== Searching For {0} =====================".format(artist))
        baseURL = self.disc.discogSearchURL
        
        url = urllib.parse.urljoin(baseURL, "{0}{1}".format("artists/", quote(artist)))
        
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,} 

        sleep(1)

        print("Downloading: {0}".format(url))

        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need

        bsdata = getHTML(data)
        
        artistDB  = {}
        
        uls = bsdata.findAll("ul", {"class": "search-results"})
        for ul in uls:
            lis = ul.findAll("li", {"class": "artist"})
            for li in lis:
                divs = li.findAll("div", {"class": "name"})
                for div in divs:
                    link     = div.find("a")
                    href     = link.attrs['href']
                    tooltip  = link.attrs['data-tooltip']
                    try:
                        from json import loads
                        tooltip = loads(tooltip)
                        artistID = tooltip['id']
                    except:
                        artistID = None
            
                    if artistDB.get(href) is None:
                        artistDB[href] = {"N": 0, "Name": artist}
                    artistDB[href]["N"] += 1
        
    
        if debug:
            print("Found {0} artists".format(len(artistDB)))
                
        iArtist = 0
        for href, hrefData in artistDB.items():
            iArtist += 1
        
            discID   = self.discogsUtils.getArtistID(href)
            url      = self.getArtistURL(href)
            savename = self.getArtistSavename(discID)

            print(iArtist,'/',len(artistDB),'\t:',discID,'\t',url)
            
            if isFile(savename):
                continue

            self.downloadArtistURL(url, savename, sleeptime=sleeptime)
                
            




    ################################################################################
    # Parse Artist Data (3)
    ################################################################################
    def parseArtistFile(ifile):
        bsdata     = getHTML(get(ifile))
        artistData = self.parse(bsdata) 
        return artistData
    
    
    def parseArtistModValExtraFiles(self, modVal, debug=False):
        force=False
        print("Parsing Artist Extra Files For ModVal {0}".format(modVal))
        artistInfo = artist()

        artistDir = self.disc.getArtistsDir()
        maxModVal = self.disc.getMaxModVal()
                    
        artistDBDir = self.disc.getArtistsDBDir()        
        
        dirVal = setDir(artistDir, str(modVal))
        dirVal = setDir(dirVal, "extra")
        files  = findExt(dirVal, ext='.p')
        
        if len(files) == 0:
            return
        print("  Found {0} extra files for ModVal {1}".format(len(files), modVal))

        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        
        if force is False:
            print("  Loaded ", end="")
            dbdata = getFile(dbname, version=3)
            print("{0} artist IDs.".format(len(dbdata)))
        else:
            print("Forcing Reloads of ModVal={0}".format(modVal))
            print("  Processing {0} files.".format(len(files)))
            dbdata = {}

        saveIt = 0
        
        nArtistMedia = {}
        
        for j,ifile in enumerate(files):
            if force is True:
                if j % 500 == 0:
                    print("\tProcessed {0}/{1} files.".format(j,len(files)))
                    
            
            info     = artistInfo.getData(ifile)
            artistID = info.ID.ID
            
            #print(artistID,'\t',sum([len(x) for x in dbdata[artistID].media.media.values()]),end="\t")

            keys = list(set(list(info.media.media.keys()) + list(dbdata[artistID].media.media.keys())))
            for k in keys:
                v = info.media.media.get(k)
                if v is None:
                    continue
                iVal  = {v2.code: v2 for v2 in v}
                dVal  = dbdata[artistID].media.media.get(k)
                if dVal is None:
                    Tretval = iVal
                    saveIt += len(iVal)
                else:
                    Tretval = {v2.code: v2 for v2 in dVal}
                    Tretval.update(iVal)
                    saveIt += len(iVal)
                dbdata[artistID].media.media[k] = list(Tretval.values())
                
            print(sum([len(x) for x in dbdata[artistID].media.media.values()]))
            
            
        if saveIt > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            print("Saving {0} new artist media to {1}".format(saveIt, savename))
            saveJoblib(data=dbdata, filename=savename, compress=True)
            
        return saveIt

    
    
    def parseArtistModValFiles(self, modVal, force=False, debug=False):        
        from os import path
        from datetime import datetime, timedelta

        print("Parsing Artist Files For ModVal {0}".format(modVal))
        artistInfo = artist()

        artistDir = self.disc.getArtistsDir()
        maxModVal = self.disc.getMaxModVal()
                    
        artistDBDir = self.disc.getArtistsDBDir()        
        
        dirVal = setDir(artistDir, str(modVal))
        files  = findExt(dirVal, ext='.p')
        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))

        
        
        ### Check for recent files
        lastModified = datetime.fromtimestamp(path.getmtime(dbname))
        now    = datetime.now()
        if force is False:
            numRecent = [ifile for ifile in files if datetime.fromtimestamp(path.getmtime(ifile)) > lastModified]
            numNew    = [ifile for ifile in files if (now-datetime.fromtimestamp(path.getmtime(ifile))).days < 1]
            if len(numRecent) == 0:
                print("  ===> Found {0} files, but there are no new files to parse so skipping.".format(len(files)))
                return 0
            else:
                print("  ===> Found {0} files. There are {1} new files to parse.".format(len(files), len(numRecent)))

                
        if force is False:
            dbdata = getFile(dbname, version=3)
        else:
            print("Forcing Reloads of ModVal={0}".format(modVal))
            print("  Processing {0} files.".format(len(files)))
            dbdata = {}

        saveIt = 0
        for j,ifile in enumerate(files):
            if force is True:
                if j % 500 == 0:
                    print("\tProcessed {0}/{1} files.".format(j,len(files)))
            artistID = getBaseFilename(ifile)
            isKnown  = dbdata.get(artistID)
            recent   = datetime.fromtimestamp(path.getctime(ifile))
            if isKnown is None or recent > lastModified:
                saveIt += 1
                info   = artistInfo.getData(ifile)
                
                if info.ID.ID != artistID:
                    print("ID From Name: {0}".format(artistID))
                    print("ID From File: {0}".format(info.ID.ID))

                    print("File: {0}".format(ifile))
                    print("Info: {0}".format(info.url.get()))
                    1/0
                
                dbdata[artistID] = info

        if saveIt > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            print("Saving {0} new artist IDs to {1}".format(saveIt, savename))
            saveJoblib(data=dbdata, filename=savename, compress=True)
            
            self.createArtistModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            self.createArtistAlbumModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            
        return saveIt
    

    def parseArtistFiles(self, force=False, debug=False):   
        
        totalSaves = 0
        maxModVal  = self.disc.getMaxModVal()
        for modVal in range(maxModVal):
            saveIt = self.parseArtistModValFiles(modVal, force=force, debug=debug)
            totalSaves += saveIt
            
        print("Saved {0} new artist IDs".format(totalSaves))      
        
        
    
    ################################################################################
    # Check ArtistDB Files
    ################################################################################ 
    def rmIDFromDB(self, artistID, modValue=None):
        if modValue is None:
            modValue  = self.discogsUtils.getDiscIDHashMod(discID=artistID, modval=self.disc.getMaxModVal())
        artistDBDir = self.disc.getArtistsDBDir()
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)
        
        saveVal = False

        if isinstance(artistID, str):
            artistID = [artistID]
        elif not isinstance(artistID, list):
            raise ValueError("Not sure what to do with {0}".format(artistID))
            
        for ID in artistID:
            try:
                del dbdata[ID]
                print("Deleted {0}".format(ID))
                saveVal = True
            except:
                print("Not there...")

            try:
                savename = self.getArtistSavename(ID)
                removeFile(savename)
                print("Removed File {0}".format(savename))
            except:
                print("Not there...")

        if saveVal:
            print("Saving {0}".format(dbname))
            saveFile(idata=dbdata, ifile=dbname)
        else:
            print("No reason to save {0}".format(dbname))

    
    def assertDBModValExtraData(self, modVal):
        
        artistDBDir = self.disc.getArtistsDBDir()
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
        dbdata  = getFile(dbname)
        nerrs = 0
        
        for artistID,artistData in dbdata.items():
            pages = artistData.pages
            if pages.more is True:
                npages = pages.pages
                artistRef = artistData.url.url
                for p in range(2, npages+1):
                    url      = self.getArtistURL(artistRef, p)
                    savename = self.getArtistSavename(artistID, p)
                    if not isFile(savename):
                        self.downloadArtistURL(url=url, savename=savename, force=True, debug=True)
                        sleep(2)
                        
            
    def assertDBModValData(self, modVal):
        
        artistDBDir = self.disc.getArtistsDBDir()
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
        dbdata  = getFile(dbname)
        nerrs = 0
        
        if self.artistIDtoRefData is None:
            self.artistIDtoRefData = self.disc.getArtistIDToRefData()
        
        dels = []
        for artistID,artistData in dbdata.items():
            pages = artistData.pages
            if pages.redo is True and False:
                artistRef = artistData.url.url
                url       = self.getArtistURL(artistRef, 1)
                savename  = self.getArtistSavename(artistID, 1)
                self.downloadArtistURL(url=url, savename=savename, force=True, debug=True)

            ID = artistData.ID.ID
            if ID != artistID:

                nerrs += 1

                if "-" in artistID:
                    print("Extra file: {0}".format(artistID))
                    continue
                else:
                    dels.append(artistID)
                    
                    rmsavename = self.getArtistSavename(artistID)


                    ## ID = artistID                    
                    refRef      = self.artistIDtoRefData.get(artistID)
                    if refRef is None:
                        raise ValueError("Ref for ID [{0}] is None!".format(artistID))
                    else:
                        print("ArtistRef:",refRef)
                        urlRef         = self.getArtistURL(refRef)
                        savenameArtRef = self.getArtistSavename(artistID)


                    ## ID = info.ID.ID
                    try:
                        info  = self.getFileData(artistID)
                    except:
                        info  = None

                    if info is not None:
                        try:
                            refIDID      = artistIDtoRefData[info.ID.ID]
                        except:
                            refIDID      = info.url.url
                        print("ArtistID: ",refIDID)
                        urlIDID      = self.getArtistURL(refIDID)
                        savenameIDID = self.getArtistSavename(info.ID.ID)
                    else:
                        refIDID      = None
                        urlIDID      = None
                        savenameIDID = None

                        
                    if isFile(rmsavename):
                        removeFile(rmsavename)


                    if isFile(savenameArtRef):
                        removeFile(savenameArtRef)
                        self.downloadArtistURL(url=urlRef, savename=savenameArtRef, force=True, debug=True)

                    if savenameArtRef != savenameIDID:
                        if isFile(savenameIDID):
                            removeFile(savenameIDID)
                            self.downloadArtistURL(url=urlIDID, savename=savenameIDID, force=True, debug=True)


                    #print(rmsavename,'\t',savenameArtID,'\t',savenameIDID)        
        
        print("Found {0} errors with modVal {1}".format(nerrs, modVal))
        
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        print("Found {0} artist IDs in {1}".format(len(dbdata), dbname))
        
        for artistID in dels:
            print("Deleting {0}".format(artistID))
            try:
                del dbdata[artistID]
            except:
                continue
            
        if len(dels) > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            print("Saving {0} artist IDs to {1}".format(len(dbdata), savename))
            saveJoblib(data=dbdata, filename=savename, compress=True)
        
        
    
    ################################################################################
    # Collect Metadata About Artists (4)
    ################################################################################
    def createArtistModValMetadata(self, modVal, db=None, debug=False):
        if db is None:
            db = self.disc.getArtistsDBModValData(modVal)
    
        artistIDMetadata = {k: [v.artist.name, v.url.url] for k,v in db.items()}
        
        for artistID,artistData in db.items():
            if artistData.profile.variations is not None:
                artistIDMetadata[artistID].append([v2.name for v2 in artistData.profile.variations])
            else:
                artistIDMetadata[artistID].append([artistData.artist.name])
        
        artistDBDir = self.disc.getArtistsDBDir()     
        savename    = setSubFile(artistDBDir, "metadata", "{0}-Metadata.p".format(modVal))
        
        print("Saving {0} new artist IDs name data to {1}".format(len(artistIDMetadata), savename))
        saveJoblib(data=artistIDMetadata, filename=savename, compress=True)
        
        
    def createArtistAlbumModValMetadata(self, modVal, db=None, debug=False):
        if db is None:
            db = self.disc.getArtistsDBModValData(modVal)
        
        artistIDMetadata = {}
        for artistID,artistData in db.items():
            artistIDMetadata[artistID] = {}
            for mediaName,mediaData in artistData.media.media.items():
                albumURLs  = {mediaValues.code: mediaValues.url for mediaValues in mediaData}
                albumNames = {mediaValues.code: mediaValues.album for mediaValues in mediaData}
                artistIDMetadata[artistID][mediaName] = [albumNames, albumURLs]
        
        artistDBDir = self.disc.getArtistsDBDir()     
        savename    = setSubFile(artistDBDir, "metadata", "{0}-MediaMetadata.p".format(modVal))
        
        print("Saving {0} new artist IDs media data to {1}".format(len(artistIDMetadata), savename))
        saveJoblib(data=artistIDMetadata, filename=savename, compress=True)

In [ ]:
#from discogsUtils import allmusicUtils
# Basic stuff
%load_ext autoreload
%autoreload
from discogsUtils import allmusicUtils

discUtils = allmusicUtils()
artistID = discUtils.getArtistID('https://www.allmusic.com/artist/bliss-mn0001514655', debug=True)
print("retval = ",artistID)

In [ ]:
s = '{"id":"MN0001498391","thumbnail":true}'

In [ ]:
from json import loads
loads(s)

In [ ]:
from ioUtils import getFile
from fsUtils import isFile
from webUtils import getHTML, isBS4
from strUtils import fixName
from math import ceil, floor

from discogsBase import discogs

class artistAMIDClass:
    def __init__(self, ID=None, err=None):
        self.ID=ID
        self.err=err
        
    def get(self):
        return self.__dict__
    
            
class artistAMURLClass:
    def __init__(self, url=None, err=None):
        self.url = url
        self.err = err
        
    def get(self):
        return self.__dict__
        
        
class artistAMNameClass:
    def __init__(self, name=None, err=None):
        self.name = name
        self.err  = err
        
    def get(self):
        return self.__dict__
    

class artistAMMediaClass:
    def __init__(self, err=None):
        self.media = {}
        self.err   = err
        
    def get(self):
        return self.__dict__
    

class artistAMMediaDataClass:
    def __init__(self, album=None, url=None, aclass=None, aformat=None, artist=None, code=None, year=None, err=None):
        self.album   = album
        self.url     = url
        self.aclass  = aclass
        self.aformat = aformat
        self.artist  = artist
        self.code    = code
        self.year    = year
        self.err     = err
        
    def get(self):
        return self.__dict__
    

class artistAMMediaAlbumClass:
    def __init__(self, url=None, album=None, aformat=None, err=None):
        self.url     = url
        self.album   = album
        self.aformat = aformat
        self.err     = err        
        
    def get(self):
        return self.__dict__

    
class artistAMMediaCountsClass:
    def __init__(self, err=None):
        self.counts = {}
        self.err    = err
        
    def get(self):
        return self.__dict__
    

class artistAMPageClass:
    def __init__(self, ppp = None, tot = None, more=None, redo=None, err=None):
        self.ppp   = ppp
        self.tot   = tot
        if isinstance(ppp, int) and isinstance(tot, int):
            self.pages = int(ceil(tot/ppp))
        else:
            self.pages = None

        self.err   = err

        self.more  = more
        self.redo  = redo
        
    def get(self):
        return self.__dict__
    

class artistAMProfileClass:
    def __init__(self, profile=None, aliases=None, members=None, sites=None, groups=None, variations=None, err=None):
        self.profile    = profile
        self.aliases    = aliases
        self.members    = members
        self.sites      = sites
        self.groups     = groups
        self.variations = variations
        self.err        = err
        
    def get(self):
        return self.__dict__
    

class artistAMURLInfo:
    def __init__(self, name=None, url=None, ID=None, err=None):
        self.name = name
        self.url  = url
        self.ID   = ID
        self.err  = err
        
    def get(self):
        return self.__dict__
        

class artistAMDataClass:
    def __init__(self, artist=None, url=None, ID=None, pages=None, profile=None, media=None, mediaCounts=None, err=None):
        self.artist      = artist
        self.url         = url
        self.ID          = ID
        self.pages       = pages
        self.profile     = profile
        self.media       = media
        self.mediaCounts = mediaCounts
        self.err         = err
        
    def get(self):
        return self.__dict__


        
class artistAM(discogs):
    def __init__(self, debug=False):
        self.debug = debug
        
    def getData(self, inputdata):
        if isinstance(inputdata, str):
            if isFile(inputdata):
                try:
                    bsdata = getHTML(getFile(inputdata))
                except:
                    try:
                        bsdata = getHTML(getFile(inputdata, version=2))
                    except:
                        raise ValueError("Cannot read artist file: {0}".format(inputdata))
            else:
                try:
                    bsdata = getHTML(inputdata)
                except:
                    raise ValueError("Not sure about string input: {0} . It is not a file".format(inputdata))
        elif isBS4(inputdata):
            bsdata = inputdata
            pass
        else:
            raise ValueError("Not sure about input type: {0}".format(type(inputdata)))

        self.bsdata = bsdata
        
        return self.parse()
        
        
            
        
    def getNamesAndURLs(self, content):
        data = []
        for ref in content.findAll("a"):
            url    = ref.attrs['href']
            name   = ref.text

            ID = self.getartistAMDiscID(url)
            data.append(artistAMURLInfo(name=name, url=url, ID=ID))
        return data





    #######################################################################################################################################
    ## Artist URL
    #######################################################################################################################################
    def getartistAMURL(self):
        result1 = self.bsdata.find("link", {"rel": "canonical"})
        result2 = self.bsdata.find("link", {"hreflang": "en"})
        if result1 and not result2:
            result = result1
        elif result2 and not result1:
            result = result2
        elif result1 and result2:
            result = result1
        else:        
            auc = artistAMURLClass(err=True)
            return auc

        if result:
            url = result.attrs["href"]
            url = url.replace("https://www.allmusic.com", "")
            if url.find("/artist/") == -1:
                url = None
                auc = artistAMURLClass(url=url, err="NoArtist")
            else:
                auc = artistAMURLClass(url=url)
        else:
            auc = artistAMURLClass(err="NoLink")

        return auc

    

    #######################################################################################################################################
    ## Artist ID
    #######################################################################################################################################                
    def getartistAMDiscID(self, suburl):
        ival = "/artist"
        if isinstance(suburl, artistAMURLClass):
            suburl = suburl.url
        if not isinstance(suburl, str):
            aic = artistAMIDClass(err="NotStr")            
            return aic

        pos = suburl.find(ival)
        if pos == -1:
            aic = artistAMIDClass(err="NotArtist")            
            return aic

        data = suburl[pos+len(ival)+1:]
        pos  = data.rfind("-")
        discIDurl = data[(pos+3):]       
        discID = discIDurl.split("/")[0]
        
        try:
            int(discID)
        except:
            aic = artistAMIDClass(err="NotInt")            
            return aic

        aic = artistAMIDClass(ID=discID)
        return aic
    
    

    #######################################################################################################################################
    ## Artist Name
    #######################################################################################################################################
    def getartistAMName(self):
        artistBios = self.bsdata.findAll("div", {"class": "artist-bio-container"})
        if len(artistBios) > 0:
            for div in artistBios:
                h1 = div.find("h1", {"class": "artist-name"})
                if h1 is not None:
                    artistName = h1.text.strip()
                    if len(artistName) > 0:
                        artist = fixName(artistName)
                        anc = artistAMNameClass(name=artist, err=None)
                    else:
                        anc = artistAMNameClass(name=artist, err="Fix")
                else:
                    anc = artistAMNameClass(err="NoH1")
        else:       
            anc = artistAMNameClass(err=True)
            return anc
        
        return anc
    
    

    #######################################################################################################################################
    ## Artist Media
    #######################################################################################################################################
    def getartistAMMediaAlbum(self, td):
        amac = artistAMMediaAlbumClass()
        for span in td.findAll("span"):
            attrs = span.attrs
            if attrs.get("class"):
                if 'format' in attrs["class"]:
                    albumformat = span.text
                    albumformat = albumformat.replace("(", "")
                    albumformat = albumformat.replace(")", "")
                    amac.format = albumformat
                    continue
            span.replaceWith("")

        ref = td.find("a")
        if ref:
            amac.url   = ref.attrs['href']
            amac.album = ref.text
        else:
            amac.err = "NoText"

        return amac
    
    
    def getartistAMMedia(self):
        amc  = artistAMMediaClass()
        name = "Albums"
        amc.media[name] = []

        tables = self.bsdata.findAll("table")
        for table in tables:
            trs = table.findAll("tr")

            header  = trs[0]
            ths     = header.findAll("th")
            headers = [x.text.strip() for x in ths]

            for tr in trs[1:]:
                tds = tr.findAll("td")

                ## Year
                key  = "Year"
                idx  = headers.index(key)
                year = tds[idx].text.strip()

                ## Title
                key   = "Album"
                idx   = headers.index(key)
                ref   = tds[idx].findAll("a")
                try:
                    refdata = ref[0]
                    url     = refdata.attrs['href']
                    album   = refdata.text.strip()
                    
                    data = url.split("/")[-1]
                    pos  = data.rfind("-")
                    discIDurl = data[(pos+3):]       
                    discID = discIDurl.split("/")[0]

                    try:
                        int(discID)
                        code = discID
                    except:
                        code = None
                        
                except:
                    print("Could not parse {0}".format(ref))

                amdc = artistAMMediaDataClass(album=album, url=url, aclass=None, aformat=None, artist=None, code=code, year=year)
                amc.media[name].append(amdc)

        return amc
    
    

    #######################################################################################################################################
    ## Artist Media Counts
    #######################################################################################################################################        
    def getartistAMMediaCounts(self):
        amcc = artistAMMediaCountsClass()
        amcc.err = "No Counts"
        return amcc
        
        results = self.bsdata.findAll("ul", {"class": "facets_nav"})
        if results is None or len(results) == 0:
            amcc.err = "No Counts"
            return amcc
            
        for result in results:
            for li in result.findAll("li"):
                ref = li.find("a")
                if ref:
                    attrs = ref.attrs
                    span = ref.find("span", {"class": "facet_count"})
                    count = None
                    if span:
                        count = span.text
                        credittype    = attrs.get("data-credit-type")
                        creditsubtype = attrs.get("data-credit-subtype")
                        if credittype and creditsubtype:
                            if amcc.counts.get(credittype) == None:
                                amcc.counts[credittype] = {}
                            if amcc.counts[credittype].get(creditsubtype) == None:
                                try:
                                    amcc.counts[credittype][creditsubtype] = int(count)
                                except:
                                    amcc.counts[credittype][creditsubtype] = count
                                    amcc.err = "Non Int"

        return amcc
    
    

    #######################################################################################################################################
    ## Artist Variations
    #######################################################################################################################################
    def getartistAMProfile(self):       
        result = self.bsdata.find("section", {"class": "basic-info"})   
        if result is None:
            apc = artistAMProfileClass(err="No Profile")
            return apc
           
        data   = {}
       
        members = result.find("div", {"class": "group-members"})
        data["Members"] = [item.text.strip() for item in members.findAll("span")]
       
        genres = result.find("div", {"class": "genre"})
        genre  = self.getNamesAndURLs(genres)
        styles = result.find("div", {"class": "styles"})
        style = self.getNamesAndURLs(genres)
        data["Profile"] = str({'genre': genre, 'style': style})
               
        apc = artistAMProfileClass(profile=data.get("Profile"), aliases=data.get("Aliases"),
                                 members=data.get("Members"), groups=data.get("In Groups"),
                                 sites=data.get("Sites"), variations=data.get("Variations"))
        return apc


    
    #######################################################################################################################################
    ## Artist Pages
    #######################################################################################################################################
    def getartistAMPages(self):
        apc   = artistAMPageClass()
        return apc
        from numpy import ceil
        bsdata = self.bsdata


        pageData = bsdata.find("div", {"class": "pagination bottom"})
        if pageData is None:
            err = "pagination bottom"
            apc = artistAMPageClass(err=err)
            return apc
        else:
            x = pageData.find("strong", {"class": "pagination_total"})
            if x is None:
                err = "pagination_total"
                apc = artistAMPageClass(err=err)
                return apc
            else:
                txt = x.text
                txt = txt.strip()
                txt = txt.replace("\n", "")
                retval = [tmp.strip() for tmp in txt.split('of')]

                try:
                    ppp   = int(retval[0].split('–')[-1])
                    tot   = int(retval[1].replace(",", ""))
                except:
                    err   = "int"
                    apc   = artistAMPageClass(err=err)
                    return apc

                if ppp < 500:
                    if tot < 25 or ppp == tot:
                        apc   = artistAMPageClass(ppp=ppp, tot=tot, redo=False, more=False)
                    else:
                        apc   = artistAMPageClass(ppp=ppp, tot=tot, redo=True, more=False)
                else:
                    if tot < 500:
                        apc   = artistAMPageClass(ppp=ppp, tot=tot, redo=False, more=False)
                    else:
                        apc   = artistAMPageClass(ppp=ppp, tot=tot, redo=False, more=True)
                        
                return apc
            
        return artistAMPageClass()



    def parse(self):
        bsdata = self.bsdata
        
        artist      = self.getartistAMName()
        url         = self.getartistAMURL()
        ID          = self.getartistAMDiscID(url)
        pages       = self.getartistAMPages()
        profile     = self.getartistAMProfile()
        mediaCounts = self.getartistAMMediaCounts()
        media       = self.getartistAMMedia()
        
        err = [artist.err, url.err, ID.err, pages.err, profile.err, mediaCounts.err, media.err]
        
        adc = artistAMDataClass(artist=artist, url=url, ID=ID, pages=pages, profile=profile, mediaCounts=mediaCounts, media=media, err=err)
        
        return adc

In [ ]:
file = "/Volumes/Music/Discog/artists-allmusic/68/0000782768.p"
artAM = artistAM()
data = artAM.getData(file)

In [ ]:
data.artist.get()

In [ ]:
data.profile.get()

In [ ]:
data.media.media["Albums"][0].get()

In [ ]:
from searchUtils import findWalkDirExt

In [ ]:
dirval = "/Volumes/Music/Discog/albums/0"

In [ ]:
files = findWalkExt(dirval, ".p")

In [ ]:
len(files)

In [ ]:
files